In [ ]:
import os
import shutil
import random
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from collections import defaultdict

# ================================
# CONFIGURAÇÕES - ALTERAR AQUI
# ================================

In [ ]:
# Caminhos dos datasets originais
CARTOON_PATH = "VCPI_individual/cartoonset100k_jpg"
CELEBA_PATH = "VCPI_individual/celebahq256_imgs"

# Pasta de destino para dataset organizado
OUTPUT_PATH = "dataset/real2cartoon"

# Número total de imagens por domínio (real/cartoon)
TOTAL_IMAGES = 30000  # Altere aqui: 1000 para testes rápidos, 30000 para treino completo

# Divisão treino/teste (0.8 = 80% treino, 20% teste)
TRAIN_SPLIT = 0.8

# ================================
# FUNÇÕES AUXILIARES
# ================================

In [ ]:
def get_all_images(path, extensions=['.jpg', '.jpeg', '.png']):
    """Obter todas as imagens de uma pasta (recursivamente)"""
    images = []
    for ext in extensions:
        images.extend(Path(path).glob(f"**/*{ext}"))
        images.extend(Path(path).glob(f"**/*{ext.upper()}"))
    return [str(img) for img in images]

def create_directory_structure(base_path):
    """Criar estrutura de pastas para CycleGAN"""
    dirs = ['trainA', 'trainB', 'testA', 'testB']
    for dir_name in dirs:
        os.makedirs(os.path.join(base_path, dir_name), exist_ok=True)
    print(f"✅ Estrutura criada em: {base_path}")

def copy_images_with_progress(source_list, dest_folder, prefix="img"):
    """Copiar imagens com barra de progresso manual"""
    os.makedirs(dest_folder, exist_ok=True)
    
    for i, src_path in enumerate(source_list):
        # Nome do arquivo de destino
        ext = os.path.splitext(src_path)[1]
        dest_name = f"{prefix}_{i:06d}{ext}"
        dest_path = os.path.join(dest_folder, dest_name)
        
        # Copiar arquivo
        shutil.copy2(src_path, dest_path)
        
        # Progresso a cada 1000 imagens
        if (i + 1) % 1000 == 0 or (i + 1) == len(source_list):
            print(f"  Copiadas: {i + 1}/{len(source_list)} imagens")
    
    return len(source_list)

def display_sample_images(dataset_path, n_samples=4):
    """Mostrar amostras de cada pasta"""
    dirs = ['trainA', 'trainB', 'testA', 'testB']
    titles = ['Real (Train)', 'Cartoon (Train)', 'Real (Test)', 'Cartoon (Test)']
    
    fig, axes = plt.subplots(4, n_samples, figsize=(15, 12))
    fig.suptitle('Amostras do Dataset Organizado', fontsize=16)
    
    for row, (dir_name, title) in enumerate(zip(dirs, titles)):
        folder_path = os.path.join(dataset_path, dir_name)
        images = get_all_images(folder_path)
        
        if len(images) >= n_samples:
            sample_images = random.sample(images, n_samples)
            for col, img_path in enumerate(sample_images):
                img = Image.open(img_path)
                axes[row, col].imshow(img)
                axes[row, col].axis('off')
                if col == 0:
                    axes[row, col].set_ylabel(title, fontsize=12)
        else:
            for col in range(n_samples):
                axes[row, col].text(0.5, 0.5, 'Sem imagens', ha='center', va='center')
                axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# ================================
# FUNÇÃO PRINCIPAL
# ================================

In [ ]:
def organize_datasets():
    """Organizar datasets na estrutura do CycleGAN"""
    
    global TOTAL_IMAGES  # Permitir modificação da variável global
    
    print("=" * 60)
    print("🎨 ORGANIZADOR DE DATASETS - REAL2CARTOON")
    print("=" * 60)
    print(f"📊 Configuração atual:")
    print(f"   • Total de imagens por domínio: {TOTAL_IMAGES:,}")
    print(f"   • Divisão treino/teste: {int(TRAIN_SPLIT*100)}% / {int((1-TRAIN_SPLIT)*100)}%")
    print(f"   • Pasta de saída: {OUTPUT_PATH}")
    print()
    
    # 1. Verificar se datasets existem
    print("🔍 1. Verificando datasets originais...")
    
    if not os.path.exists(CARTOON_PATH):
        print(f"❌ ERRO: Não encontrado {CARTOON_PATH}")
        return False
    
    if not os.path.exists(CELEBA_PATH):
        print(f"❌ ERRO: Não encontrado {CELEBA_PATH}")
        return False
    
    print(f"✅ Cartoon dataset: {CARTOON_PATH}")
    print(f"✅ CelebA dataset: {CELEBA_PATH}")
    
    # 2. Contar imagens disponíveis
    print("\n📈 2. Contando imagens disponíveis...")
    
    cartoon_images = get_all_images(CARTOON_PATH)
    celeba_images = get_all_images(CELEBA_PATH)
    
    print(f"   • Cartoons encontrados: {len(cartoon_images):,}")
    print(f"   • Rostos reais encontrados: {len(celeba_images):,}")
    
    # Verificar se há imagens suficientes
    if len(cartoon_images) < TOTAL_IMAGES:
        print(f"⚠️  AVISO: Só há {len(cartoon_images)} cartoons (pedido: {TOTAL_IMAGES})")
        TOTAL_IMAGES = len(cartoon_images)
    
    if len(celeba_images) < TOTAL_IMAGES:
        print(f"⚠️  AVISO: Só há {len(celeba_images)} rostos reais (pedido: {TOTAL_IMAGES})")
        TOTAL_IMAGES = min(TOTAL_IMAGES, len(celeba_images))
    
    print(f"✅ Usando {TOTAL_IMAGES:,} imagens de cada tipo")
    
    # 3. Preparar divisões
    print(f"\n🎯 3. Preparando divisões...")
    
    train_size = int(TOTAL_IMAGES * TRAIN_SPLIT)
    test_size = TOTAL_IMAGES - train_size
    
    print(f"   • Treino: {train_size:,} imagens por domínio")
    print(f"   • Teste: {test_size:,} imagens por domínio")
    
    # Embaralhar e dividir
    random.shuffle(cartoon_images)
    random.shuffle(celeba_images)
    
    # Seleções
    cartoon_train = cartoon_images[:train_size]
    cartoon_test = cartoon_images[train_size:train_size + test_size]
    
    celeba_train = celeba_images[:train_size]
    celeba_test = celeba_images[train_size:train_size + test_size]
    
    # 4. Criar estrutura de pastas
    print(f"\n📁 4. Criando estrutura de pastas...")
    create_directory_structure(OUTPUT_PATH)
    
    # 5. Copiar imagens
    print(f"\n📋 5. Copiando imagens...")
    
    print("🎨 Copiando cartoons para treino...")
    copy_images_with_progress(cartoon_train, os.path.join(OUTPUT_PATH, "trainB"), "cartoon")
    
    print("🎨 Copiando cartoons para teste...")
    copy_images_with_progress(cartoon_test, os.path.join(OUTPUT_PATH, "testB"), "cartoon")
    
    print("👤 Copiando rostos reais para treino...")
    copy_images_with_progress(celeba_train, os.path.join(OUTPUT_PATH, "trainA"), "real")
    
    print("👤 Copiando rostos reais para teste...")
    copy_images_with_progress(celeba_test, os.path.join(OUTPUT_PATH, "testA"), "real")
    
    # 6. Verificação final
    print(f"\n✅ 6. Verificação final...")
    
    structure = {}
    for folder in ['trainA', 'trainB', 'testA', 'testB']:
        path = os.path.join(OUTPUT_PATH, folder)
        count = len(get_all_images(path))
        structure[folder] = count
        print(f"   • {folder}: {count:,} imagens")
    
    total_copied = sum(structure.values())
    expected = TOTAL_IMAGES * 2  # real + cartoon
    
    print(f"\n🎉 Dataset organizado com sucesso!")
    print(f"   • Total copiado: {total_copied:,} imagens")
    print(f"   • Esperado: {expected:,} imagens")
    print(f"   • Status: {'✅ OK' if total_copied == expected else '⚠️ VERIFICAR'}")
    print(f"   • Localização: {os.path.abspath(OUTPUT_PATH)}")
    
    return True

In [ ]:
def show_dataset_stats():
    """Mostrar estatísticas do dataset organizado"""
    if not os.path.exists(OUTPUT_PATH):
        print("❌ Dataset ainda não foi organizado. Execute organize_datasets() primeiro.")
        return
    
    print("\n" + "=" * 40)
    print("📊 ESTATÍSTICAS DO DATASET")
    print("=" * 40)
    
    for folder in ['trainA', 'trainB', 'testA', 'testB']:
        path = os.path.join(OUTPUT_PATH, folder)
        if os.path.exists(path):
            count = len(get_all_images(path))
            label = {'trainA': 'Rostos Reais (Treino)', 
                    'trainB': 'Cartoons (Treino)',
                    'testA': 'Rostos Reais (Teste)', 
                    'testB': 'Cartoons (Teste)'}[folder]
            print(f"{label:.<25} {count:>6,} imagens")
    
    print("\n🖼️  Mostrando amostras...")
    display_sample_images(OUTPUT_PATH)

# ================================
# EXECUÇÃO
# ================================

In [ ]:
organize_datasets()